In [2]:
import polars as pl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import polars as pl
import scipy.stats as stats
import scipy.sparse as sps
import gc
import tqdm
from polimi.utils._custom import save_sparse_csr

<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
2024-05-22 13:13:52.490228: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-22 13:13:54.962814: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
articles = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/articles.parquet')

behaviors_train = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/train/behaviors.parquet')
history_train = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/train/history.parquet')

behaviors_val = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/validation/behaviors.parquet')
history_val = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/validation/history.parquet')

contrastive_vector_2 = pl.read_parquet('/home/ubuntu/dataset/Ekstra_Bladet_contrastive_vector/contrastive_vector.parquet')
w_2_vec = pl.read_parquet('/home/ubuntu/dataset/Ekstra_Bladet_word2vec/document_vector.parquet')
roberta = pl.read_parquet('/home/ubuntu/dataset/FacebookAI_xlm_roberta_base/xlm_roberta_base.parquet')
google_bert = pl.read_parquet('/home/ubuntu/dataset/google_bert_base_multilingual_cased/bert_base_multilingual_cased.parquet')
distilbert = pl.read_parquet('/home/ubuntu/dataset/distilbert_title_embedding.parquet')
kenneth = pl.read_parquet('/home/ubuntu/dataset/kenneth_embedding.parquet')
emotions = pl.read_parquet('/home/ubuntu/dataset/emotions_embedding.parquet')



In [4]:
from polimi.utils._urm import build_user_id_mapping

In [30]:
user_mapping = build_user_id_mapping(history_train.vstack(history_val))

In [31]:
user_mapping

user_index,user_id
u32,u32
0,10068
1,10200
2,10201
3,10385
4,10623
…,…
18822,2590039
18823,2590054
18824,2590471


In [94]:
associations = {
    'contrastive_vector' : contrastive_vector_2,
    'document_vector': w_2_vec,
    'google-bert/bert-base-multilingual-cased': google_bert,
    'FacebookAI/xlm-roberta-base': roberta,
    'title_embedding': distilbert,
    'kenneth_title+subtitle': kenneth,
    'emotion_scores': emotions
    
    
}

In [93]:
from polimi.utils._embeddings import  _build_mean_user_embeddings


In [143]:
emotions = _build_mean_user_embeddings(df = None, history=history_train.vstack(history_val), embeddings=emotions,emb_type='contrastive')[0]

Building user embeddings...


31it [00:02, 12.09it/s]                        


In [136]:
emotions

user_id,mean_user_embedding
u32,list[f32]
1514775,"[-0.011782, 0.021622, … 0.026789]"
546883,"[-0.004247, 0.024449, … 0.020094]"
1561164,"[-0.010688, 0.019372, … 0.024208]"
1567717,"[-0.010414, 0.035404, … 0.023648]"
1776014,"[-0.008001, 0.025873, … 0.025874]"
…,…
602558,"[0.009478, 0.010582, … 0.033292]"
764969,"[-0.045113, -0.034643, … 0.02465]"
2028720,"[-0.031022, 0.04619, … -0.004195]"


In [144]:
name =emotions

In [145]:
UCM_dataframe = name.join(history_train.vstack(history_val), on='user_id').select('user_id','mean_user_embedding')\
                .with_columns(
                    pl.col('mean_user_embedding').apply(lambda lst : list(range(len(lst)))).alias("indici")  
                )\
                .explode(['mean_user_embedding','indici'])\
                .rename({'indici': 'feature_id'})\
                .join(user_mapping, on='user_id')\
                .drop('user_id')

UCM_dataframe

/tmp/ipykernel_282877/159819214.py:3: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col('mean_user_embedding').apply(lambda lst : list(range(len(lst)))).alias("indici")
/tmp/ipykernel_282877/159819214.py:2: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  .with_columns(


mean_user_embedding,feature_id,user_index
f32,i64,u32
0.01681,0,31
0.051164,1,31
0.386025,2,31
0.139715,3,31
0.181304,4,31
…,…,…
0.070915,1,7030
0.348788,2,7030
0.088983,3,7030


In [146]:
n_users = UCM_dataframe.select('user_index').n_unique()
print(f'n_users:{n_users}')
n_features = UCM_dataframe.select('feature_id').n_unique()
print(f'num_features: {n_features}')

n_users:18827
num_features: 6


In [147]:
UCM = sps.csr_matrix((UCM_dataframe['mean_user_embedding'].to_numpy(), 
                          (UCM_dataframe["user_index"].to_numpy(), UCM_dataframe["feature_id"].to_numpy())),
                        shape = (n_users, n_features))

In [148]:
UCM

<18827x6 sparse matrix of type '<class 'numpy.float32'>'
	with 322806 stored elements in Compressed Sparse Row format>

In [149]:
save_sparse_csr(path=Path(f'/home/ubuntu/recsys2024/ucm/recsys/small/emotions.npz'),array=UCM)

File saved at: /home/ubuntu/recsys2024/ucm/recsys/small/emotions.npz


In [86]:
from RecSys_Course_AT_PoliMi.Recommenders.KNN.UserKNNCBFRecommender import UserKNNCBFRecommender

In [87]:
from polimi.utils._custom import load_sparse_csr

URM_train = load_sparse_csr(Path('/home/ubuntu/recsys2024/urm/recsys/small/URM_train.npz'))
URM_validation_train = load_sparse_csr(Path('/home/ubuntu/recsys2024/urm/recsys/small/URM_validation_train.npz'))
URM_validation = load_sparse_csr(Path('/home/ubuntu/recsys2024/urm/recsys/small/URM_validation.npz'))
URM_validation_validation = load_sparse_csr(Path('/home/ubuntu/recsys2024/urm/recsys/small/URM_validation_validation.npz'))


File loaded at: /home/ubuntu/recsys2024/urm/recsys/small/URM_train.npz
File loaded at: /home/ubuntu/recsys2024/urm/recsys/small/URM_validation_train.npz
File loaded at: /home/ubuntu/recsys2024/urm/recsys/small/URM_validation.npz
File loaded at: /home/ubuntu/recsys2024/urm/recsys/small/URM_validation_validation.npz


In [88]:
UCM.shape

(18827, 768)

In [89]:
URM_train.shape

(18827, 20738)

In [90]:
knn_ucm = UserKNNCBFRecommender(URM_train=URM_train, UCM_train=UCM)
knn_ucm.fit()

UserKNNCBFRecommender: URM Detected 3684 (19.6%) users with no interactions.
UserKNNCBFRecommender: URM Detected 11952 (57.6%) items with no interactions.
Compute_Similarity: detected dense matrix
Similarity column 18827 (100.0%), 160.57 column/sec. Elapsed time 1.95 min


In [91]:
from RecSys_Course_AT_PoliMi.Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation_train, cutoff_list=[100])

EvaluatorHoldout: Ignoring 3684 (19.6%) Users that have less than 1 test interactions


In [92]:
result_df, _ = evaluator_validation.evaluateRecommender(knn_ucm)
result_df.loc[100]["NDCG"]

EvaluatorHoldout: Processed 15143 (100.0%) in 18.38 sec. Users per second: 824


0.0005440506252408157